In [1]:
!pip install --upgrade scikit-image

     |████████████████████████████████| 26.5MB 208kB/s eta 0:00:01
  Found existing installation: scikit-image 0.13.1
    Uninstalling scikit-image-0.13.1:
      Successfully uninstalled scikit-image-0.13.1
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import efficientnet.keras as efn 

model = efn.EfficientNetB0(weights='imagenet')  # or weights='noisy-student'

Using TensorFlow backend.














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

21831680/21826536 [==============================] - 4s 0us/step


In [3]:
import tensorflow as tf
import tensorflow.keras

print(tf.__version__)
print(tensorflow.keras.__version__)

1.15.2
2.2.4-tf
